In [15]:
import gensim
#import fasttext as ft
import re
import itertools
import collections 
import nltk
import pandas as pd
import numpy as np
import string

In [16]:
articles = pd.read_csv('articles1.csv')
#articles = articles1.iloc[:100,:]

In [17]:
import time
start = time.time()

In [18]:
#topic generator
def topic_generation(x):
    doc_complete = nltk.sent_tokenize(x)
    stop = set(nltk.corpus.stopwords.words('english'))
    exclude = set(string.punctuation) 
    lemma = nltk.stem.wordnet.WordNetLemmatizer()
    def clean(doc):
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalized

    doc_clean = [clean(doc).split() for doc in doc_complete]
    dictionary = gensim.corpora.Dictionary(doc_clean)

    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    Lda = gensim.models.ldamodel.LdaModel

    # Running and Trainign LDA model on the document term matrix.
    try:
        ldamodel = Lda(doc_term_matrix, num_topics=len(doc_complete), id2word = dictionary, passes=20)
        topic_list = []

        topics = ldamodel.show_topics(num_topics = len(doc_complete), num_words = 1)
        for i in range(len(topics)):
            if topics[i][1][7:-1] not in topic_list:
                topic_list.append(topics[i][1][7:-1])
        return topic_list

    except ValueError:
        pass
   
articles['topics'] = articles.content.map(lambda x:topic_generation(x))

/home/ian/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
/home/ian/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:582: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words


In [19]:
end = time.time()
print('Total time: ', end-start)
articles.to_csv('articles_with_topics.csv', index =False)

Total time:  17928.932272434235
